
# Predicción de Retrasos de Vuelos en la Industria Aérea ✈️

**Objetivo:** Construir un modelo que **prediga si un vuelo sufrirá un retraso de llegada mayor a 15 minutos** (`RETRASADO_LLEGADA` = 1).  
Este notebook sigue la estructura de `machine_learning.ipynb`, con secciones de carga de datos, EDA, preparación, entrenamiento con **LightGBM**, evaluación y conclusiones.

**Dataset de entrada:** `data/processed/flights_clean.csv` (resultado del pipeline de limpieza e ingeniería de características).  
**Tamaño esperado:** ~5.2M filas (podría requerir >8GB RAM).


## 1. Importaciones y configuración

In [ ]:

import os, time, math, json, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix

# LightGBM
try:
    import lightgbm as lgb
except ImportError as e:
    raise ImportError("LightGBM no está instalado. Instala con: pip install lightgbm") from e

# Configuración visual
plt.rcParams["figure.figsize"] = (10, 6)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)


## 2. Carga de datos

In [ ]:

DATA_PATH = os.path.join(os.path.dirname(os.path.abspath("__file__")), "data", "processed", "flights_clean.csv")
print("Cargando:", DATA_PATH)

t0 = time.time()
v = pd.read_csv(DATA_PATH, low_memory=False)
t1 = time.time()

print(f"✅ Datos cargados: {v.shape[0]:,} filas × {v.shape[1]} columnas en {t1 - t0:.2f}s")
v.head()


## 3. Inspección rápida de columnas

In [ ]:

v.info(memory_usage='deep', show_counts=True)


## 4. Distribución de retrasos (llegada > 15 min)

In [ ]:

assert "RETRASADO_LLEGADA" in v.columns, "No existe la columna RETRASADO_LLEGADA en el dataset limpio."

conteo = v["RETRASADO_LLEGADA"].value_counts().sort_index()
porc = (conteo / conteo.sum() * 100).round(2)

print("📊 Distribución de vuelos según retraso en llegada (>15 min):\n")
print(f"A tiempo (0): {conteo.get(0,0):,} vuelos ({porc.get(0,0):.2f}%)")
print(f"Retrasados (1): {conteo.get(1,0):,} vuelos ({porc.get(1,0):.2f}%)")


## 5. Selección de variables (features)


Usaremos variables **categóricas y de tiempo** ya generadas en el pipeline:

- `AIRLINE`, `ORIGIN_AIRPORT`, `DESTINATION_AIRPORT` (categóricas)
- `MONTH`, `DAY_OF_WEEK` (tiempo)
- Codificación cíclica: `SALIDA_SIN`, `SALIDA_COS` (derivadas de la hora programada de salida)


In [ ]:

target = "RETRASADO_LLEGADA"
features = [
    "AIRLINE",
    "ORIGIN_AIRPORT",
    "DESTINATION_AIRPORT",
    "MONTH",
    "DAY_OF_WEEK",
    "SALIDA_SIN",
    "SALIDA_COS"
]

missing = [c for c in features + [target] if c not in v.columns]
if missing:
    raise ValueError(f"Faltan columnas requeridas: {missing}")

X = v[features].copy()
y = v[target].astype(int).copy()

# Liberar memoria del dataframe original si es necesario
del v
gc.collect()

X.head()


## 6. Codificación de variables categóricas


Para eficiencia con >5M de filas, usamos **Label Encoding** para `AIRLINE`, `ORIGIN_AIRPORT`, `DESTINATION_AIRPORT`.  
LightGBM maneja bien etiquetas enteras y permite splits por categoría, especialmente cuando las variables no son ordinales reales.


In [ ]:

categorical_cols = ["AIRLINE", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT"]
encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    encoders[col] = le

# Guardar encoders en memoria (opcional: persistir a disco si se desea)
print("✅ Categóricas codificadas:", categorical_cols)


## 7. División Train/Test (estratificada)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print(f"Train: {X_train.shape}, Test: {X_test.shape}")


## 8. Entrenamiento: LightGBM (class_weight='balanced')

In [ ]:

params = dict(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=8,
    num_leaves=63,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    class_weight="balanced",  # ✅ Compensa desbalance (18/82 aprox.)
    n_jobs=-1
)

model = lgb.LGBMClassifier(**params)

t0 = time.time()
model.fit(X_train, y_train)
t1 = time.time()

print(f"✅ Modelo entrenado en {t1 - t0:.2f}s")


## 9. Evaluación del modelo

In [ ]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
roc = roc_auc_score(y_test, y_proba)

print("Classification report:\n", classification_report(y_test, y_pred, digits=4))
print(f"Accuracy:   {acc:.4f}")
print(f"Precision:  {prec:.4f}")
print(f"Recall:     {rec:.4f}")
print(f"F1-score:   {f1:.4f}")
print(f"ROC-AUC:    {roc:.4f}")


### Curva ROC

In [ ]:

fpr, tpr, thr = roc_curve(y_test, y_proba)

plt.figure()
plt.plot(fpr, tpr, label=f"ROC-AUC = {roc:.3f}")
plt.plot([0,1], [0,1], linestyle="--", color="grey")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Curva ROC - Retrasos de Llegada")
plt.legend()
plt.grid(alpha=0.3)
plt.show()


### Matriz de confusión

In [ ]:

cm = confusion_matrix(y_test, y_pred)
cm


## 10. Importancia de variables

In [ ]:

ax = lgb.plot_importance(model, max_num_features=20, importance_type="gain")
plt.title("Importancia de variables (LightGBM)")
plt.tight_layout()
plt.show()


## 11. Función de predicción (para integración futura con API)

In [ ]:

def preparar_entrada(airline, origin, destination, month, day_of_week, scheduled_hour, scheduled_minute):
    """
    Prepara un diccionario con las features necesarias para predicción.
    - scheduled_hour: 0-23
    - scheduled_minute: 0-59
    """
    minuto_dia = scheduled_hour * 60 + scheduled_minute
    salida_sin = math.sin(2 * math.pi * minuto_dia / (24*60))
    salida_cos = math.cos(2 * math.pi * minuto_dia / (24*60))
    row = {
        "AIRLINE": encoders["AIRLINE"].transform([str(airline)])[0],
        "ORIGIN_AIRPORT": encoders["ORIGIN_AIRPORT"].transform([str(origin)])[0],
        "DESTINATION_AIRPORT": encoders["DESTINATION_AIRPORT"].transform([str(destination)])[0],
        "MONTH": month,
        "DAY_OF_WEEK": day_of_week,
        "SALIDA_SIN": salida_sin,
        "SALIDA_COS": salida_cos
    }
    return row

def predecir_probabilidad_delay(sample_dict):
    """Recibe un diccionario de features y devuelve probabilidad de retraso en llegada (>15 min)."""
    df = pd.DataFrame([sample_dict])[list(model.feature_name_)]
    proba = model.predict_proba(df)[:, 1][0]
    return float(proba)

# Ejemplo de uso:
ejemplo = preparar_entrada("AA", "JFK", "LAX", 5, 4, 14, 30)
print("Probabilidad de retraso (ejemplo):", round(predecir_probabilidad_delay(ejemplo), 4))


## 12. Conclusiones y siguientes pasos


- El modelo **LightGBM** con `class_weight="balanced"` maneja correctamente el desbalance (~18% retrasos).
- Las variables de ubicación (aeropuerto y aerolínea) y la codificación cíclica de la hora suelen aportar poder predictivo.
- Para producción:
  - serializar `model` y `encoders` con `joblib`,
  - crear un endpoint `/flights/predict-delay` con FastAPI,
  - validar en datos recientes y monitorear métricas.

**Mejoras posibles:**
- Agregar variable de **distancia Haversine**.
- Agregar **mes/día** como seno/coseno (estacionalidad).
- Regularización y **búsqueda de hiperparámetros** (Optuna).


## 13. (Opcional) Guardado de modelo y encoders

In [ ]:

# from joblib import dump
# os.makedirs("models", exist_ok=True)
# dump(model, "models/lgbm_delay_model.joblib")
# dump(encoders, "models/label_encoders.joblib")
# print("✅ Modelo y encoders guardados en carpeta models/")
